In [19]:
import cv2
import numpy as np
from scipy.ndimage import zoom
from PIL import Image

import os
import logging
import subprocess
import pandas as pd

from matplotlib import pyplot as plt

def convert_to_frame(input_video_path):
    frames_list = []
    capture = cv2.VideoCapture(input_video_path)    
    fps = int(capture.get(cv2.CAP_PROP_FPS))
    frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    for noFrames in range(frame_count):
        ret, frame = capture.read()
        if not ret:
            break

        #img = Image.fromarray(frame)
        #resized_img = img.resize((299,299), Image.LANCZOS)
        #frame = np.array(resized_img)
        
        resizedFrame = cv2.resize(frame, (299,299), interpolation=cv2.INTER_LANCZOS4)
        frames_list.append(resizedFrame)

    capture.release()
    return frames_list


def convertListToGrayScale(videoFrames):
    grayImageList = []
    for frame in videoFrames:
        grayImage = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        grayImageList.append(grayImage)
    return grayImageList

    

def performThreshold(diffFrame):
    mask = (diffFrame < 13) & (diffFrame > -13)
    diffFrame[mask] = 0
    return diffFrame

def binarize(noiseless_image):
    mean = np.mean(noiseless_image)
    maskZeroes = (noiseless_image < mean)
    maskOnes = (noiseless_image >= mean)
    binarizedImage = noiseless_image
    binarizedImage[maskZeroes] = 0
    binarizedImage[maskOnes] = 1
    return binarizedImage

def accumulateFrames(framesList):
    accumulatedFrame = framesList[0]

    for i in range(1, len(framesList)):
        accumulatedFrame += framesList[i]
    
    return accumulatedFrame

def applyForwardDiff(grayImagesList):
    listOfFrames = []

    for index in range(len(grayImagesList) - 1):
        forward_ImageDiff = grayImagesList[index] - grayImagesList[index + 1]
        forward_ImageDiff = performThreshold(forward_ImageDiff)
        binarizedImage = binarize(forward_ImageDiff)        
        listOfFrames.append(binarizedImage)
    
    return accumulateFrames(listOfFrames)

def applyBackwardDiff(grayImagesList):
    listOfFrames = []

    for index in range(1, len(grayImagesList)):
        backward_ImageDiff = grayImagesList[index] - grayImagesList[index - 1]
        backward_ImageDiff = performThreshold(backward_ImageDiff)
        binarizedImage = binarize(backward_ImageDiff) 
        listOfFrames.append(binarizedImage)
    
    return accumulateFrames(listOfFrames)

def applyBiDirectionalDiff(grayImagesList):
    listOfFrames = []

    for index in range(1, len(grayImagesList)-1):
        average_frame = np.mean([grayImagesList[index - 1], grayImagesList[index + 1]], axis=0)
        bidirectional_ImageDiff = grayImagesList[index] - average_frame
        bidirectional_ImageDiff = performThreshold(bidirectional_ImageDiff)
        binarizedImage = binarize(bidirectional_ImageDiff)
        listOfFrames.append(binarizedImage)
    
    return accumulateFrames(listOfFrames)

def applyATD(frames, output_directory, index):
    forwardATD = applyForwardDiff(frames)
    backwardATD = applyBackwardDiff(frames)
    bidirectATD = applyBiDirectionalDiff(frames)
    
    wordFolder = os.path.join(output_directory, f'word_{index}')

    if not os.path.exists(wordFolder):
        os.makedirs(wordFolder)

    stackedImage = np.dstack([forwardATD, backwardATD, bidirectATD]).astype(np.uint8)
    stackedImage = np.clip(stackedImage.astype(np.int16) + 128, 0, 255).astype(np.uint8)
    stackedImage_path = os.path.join(wordFolder, f"{index}.png")
    cv2.imwrite(stackedImage_path, stackedImage)


list_of_frames = convert_to_frame(r"C:\Users\salma\Downloads\Testing Sample\01_0001_(10_03_21_20_37_10)_c.mp4")                             
frames = convertListToGrayScale(list_of_frames)

path = os.path.join(r"C:\Users\salma\Downloads\04_0010_(13_03_21_00_11_54)_c")

noWords = 2
split = len(frames) // noWords
for n in range(noWords): 
    index = split * n
    if n == noWords-1:
        applyATD(frames[index:], path, n)
    else:
        applyATD(frames[index: index + split], path, n)